# Lab 03: TensorFlow and PyTorch




## TensorFlow Implementation

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255
x_test = x_test / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),  # Number of hidden neurons
    tf.keras.layers.Dense(10, activation='softmax')  # Number of output neurons
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',       # Name of the loss function
              metrics=['accuracy'])

start = time.time()
model.fit(x_train, y_train, epochs=5)
end = time.time()
print(f"TF Training time: {end-start:.2f} seconds")       # Outputs the training time
model.evaluate(x_test, y_test)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8602 - loss: 0.5057
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9560 - loss: 0.1529
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9685 - loss: 0.1084
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9745 - loss: 0.0857
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9799 - loss: 0.0679
TF Training time: 15.76 seconds
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9675 - loss: 0.1030  


[0.09240816533565521, 0.9714000225067139]

## Convert TensorFlow model to TFLite

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmplh2opi__'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  138628117417424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138628117420112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138628117420304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138628117420496: TensorSpec(shape=(), dtype=tf.resource, name=None)


## PyTorch Implementation

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([transforms.ToTensor(), transforms.Lambda(lambda x: x.view(-1))])
train_loader = DataLoader(datasets.MNIST(root='./data', train=True, transform=transform, download=True), batch_size=32)
test_loader = DataLoader(datasets.MNIST(root='./data', train=False, transform=transform, download=True), batch_size=1000)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 64)    # Input and output size
        self.fc2 = nn.Linear(64, 10)    # Input and output size
    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

model = Net()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

start = time.time()
for epoch in range(5):
    for x, y in train_loader:
        optimizer.zero_grad()
        pred = model(x)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
end = time.time()
print(f"PyTorch Training time: {end - start:.2f} seconds")

model.eval()
correct = 0
with torch.no_grad():
    for x, y in test_loader:
        output = model(x)
        pred = output.argmax(1)
        correct += (pred == y).sum().item()
print(f"Test accuracy: {correct / len(test_loader.dataset):.4f}")

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.3MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 495kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.61MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 10.7MB/s]


PyTorch Training time: 37.03 seconds
Test accuracy: 0.9682


## Convert PyTorch model to ONNX

In [ ]:
# Installing ONNX
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 140.7 MB/s eta 0:00:00


In [ ]:
dummy_input = torch.randn(1, 784)
torch.onnx.export(model, dummy_input, "model.onnx",
                  input_names=["input"], output_names=["output"])

## TensorFlow custom training loop using tf.GradientTape

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0   # Normalization factor
x_test = x_test / 255.0   # Normalization factor
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Prepare datasets
batch_size = 32         # Batch size as in first TF example
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Defining model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),    # Number of neurons and activation
    tf.keras.layers.Dense(10, activation='softmax')     # Number of neurons and activation
])

# Defining loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss = loss_fn(y_batch, logits)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_acc_metric.update_state(y_batch, logits)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch+1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")



Epoch 1/5
Step 0, Loss: 2.3997, Accuracy: 0.0312
Step 100, Loss: 0.4086, Accuracy: 0.7222
Step 200, Loss: 0.2637, Accuracy: 0.7965
Step 300, Loss: 0.2558, Accuracy: 0.8258
Step 400, Loss: 0.5424, Accuracy: 0.8448
Step 500, Loss: 0.3789, Accuracy: 0.8582
Step 600, Loss: 0.4689, Accuracy: 0.8681
Step 700, Loss: 0.4306, Accuracy: 0.8761
Step 800, Loss: 0.1280, Accuracy: 0.8826
Step 900, Loss: 0.1377, Accuracy: 0.8876
Step 1000, Loss: 0.2222, Accuracy: 0.8921
Step 1100, Loss: 0.3313, Accuracy: 0.8961
Step 1200, Loss: 0.3341, Accuracy: 0.8988
Step 1300, Loss: 0.1639, Accuracy: 0.9018
Step 1400, Loss: 0.2513, Accuracy: 0.9042
Step 1500, Loss: 0.1224, Accuracy: 0.9071
Step 1600, Loss: 0.0844, Accuracy: 0.9094
Step 1700, Loss: 0.1984, Accuracy: 0.9120
Step 1800, Loss: 0.2663, Accuracy: 0.9138
Training Accuracy for epoch 1: 0.9154

Epoch 2/5
Step 0, Loss: 0.3654, Accuracy: 0.9375
Step 100, Loss: 0.1198, Accuracy: 0.9573
Step 200, Loss: 0.0320, Accuracy: 0.9558
Step 300, Loss: 0.2453, Accuracy:

## Performance Otimization with Graph Execution using @tf.function

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0   # Normalization factor
x_test = x_test / 255.0   # Normalization factor
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Prepare datasets
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Defining model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),    #  Number of neurons and activation
    tf.keras.layers.Dense(10, activation='softmax')     # Number of neurons and activation
])

# Defining loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

@tf.function  # compiling the function into a graph
def train_step(x_batch, y_batch):
    with tf.GradientTape() as tape:
        logits = model(x_batch, training=True)
        loss = loss_fn(y_batch, logits)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_acc_metric.update_state(y_batch, logits)
    return loss

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        loss = train_step(x_batch, y_batch)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch+1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")



Epoch 1/5
Step 0, Loss: 2.3811, Accuracy: 0.0938
Step 100, Loss: 0.5243, Accuracy: 0.7410
Step 200, Loss: 0.3478, Accuracy: 0.8078
Step 300, Loss: 0.2486, Accuracy: 0.8394
Step 400, Loss: 0.5382, Accuracy: 0.8538
Step 500, Loss: 0.3831, Accuracy: 0.8656
Step 600, Loss: 0.4643, Accuracy: 0.8754
Step 700, Loss: 0.2827, Accuracy: 0.8825
Step 800, Loss: 0.0854, Accuracy: 0.8873
Step 900, Loss: 0.4231, Accuracy: 0.8919
Step 1000, Loss: 0.4092, Accuracy: 0.8955
Step 1100, Loss: 0.1223, Accuracy: 0.8983
Step 1200, Loss: 0.4102, Accuracy: 0.9006
Step 1300, Loss: 0.2002, Accuracy: 0.9036
Step 1400, Loss: 0.1474, Accuracy: 0.9059
Step 1500, Loss: 0.0829, Accuracy: 0.9082
Step 1600, Loss: 0.2678, Accuracy: 0.9105
Step 1700, Loss: 0.2113, Accuracy: 0.9124
Step 1800, Loss: 0.3125, Accuracy: 0.9141
Training Accuracy for epoch 1: 0.9151

Epoch 2/5
Step 0, Loss: 0.1990, Accuracy: 0.9688
Step 100, Loss: 0.1340, Accuracy: 0.9502
Step 200, Loss: 0.3113, Accuracy: 0.9485
Step 300, Loss: 0.3668, Accuracy: